# `019` Byte-pair encoding tokenization

Requirements: 016 Transformers

Even though transformers are powerful and severely speed up the training process thanks to parallelization, they are still limited by the maximum sequence length. Because the attention matrix is a square matrix of size `sequence_length x sequence_length`, the memory complexity of the model is $O(L^2)$, where $L$ is the sequence length. This means that the model can only handle sequences of a certain length.

One of the most typical ways to handle this limitation is by adjusting how the input is tokenized. So far, we have been using a simple character-level tokenization, in which a sentence like "I like trains" would have $12$ tokens, and our total vocabulary was the set of all English characters and digits plus some special tokens, which in total is $\approx 70$ tokens.

If instead, we decided to use a word-level tokenization, the sentence "I like trains" would have $3$ tokens, and our vocabulary would be the set of all English words, which is $\approx 200,000$ tokens. This would allow us to handle much longer sequences, but would create a huge embedding table, which would be very slow to train, and since many words are not very frequent, the embeddings would not be very good.

This is why most modern models use a subword-level tokenization, which is a middle ground between character-level and word-level tokenization. The most popular subword tokenization algorithm is Byte-pair encoding (BPE). BPE, like most tokenization algorithms starts by converting the input text into a list of bytes, typically using UTF-8 encoding because it's [the best we got](https://www.reedbeta.com/blog/programmers-intro-to-unicode/). UTF-8 is a variable-length encoding, which means that each character can be represented by a different number of bytes.

In [1]:
characters = 'aá🤯'
for c in characters:
	print(f'Character {c} is represented by bytes {list(c.encode("utf-8"))}')

Character a is represented by bytes [97]
Character á is represented by bytes [195, 161]
Character 🤯 is represented by bytes [240, 159, 164, 175]


Now that we have converted our text into a sequence of numbers $[0, 255]$ representing the bytes, the BPE algorithm is very simple: it just scans all pairs of elements keeping track of the most common pair of elements and merging them into a new element. This process is repeated as many times as necessary.

Let's illustrate it over the dataset of Spanish novels.

In [2]:
# !pip install regex
from regex import compile as regex_compile
from time import time

In [3]:
with open('custom-data/spanish-novels.txt', encoding='utf-8') as fp:
	text = fp.read()

print(f'In total, there are {len(text.encode("utf-8")):,} UTF-8 bytes in the file.')

In total, there are 19,979,810 UTF-8 bytes in the file.


Let's create a method that, given a sequence of elements, a pair to look for, and a new element, replaces all occurrences of the pair with the new element.

In [4]:
def merge(sequence, pair, replacement):
	res = []
	i = 0
	while i < len(sequence):
		if i < len(sequence) - 1 and sequence[i] == pair[0] and sequence[i + 1] == pair[1]:
			res.append(replacement)
			i += 2
		else:
			res.append(sequence[i])
			i += 1
	return res

merge('How are you?', (' ', 'a'), 'X')

['H', 'o', 'w', 'X', 'r', 'e', ' ', 'y', 'o', 'u', '?']

Now, before continuing with the tokenization method, there is a constraint we want to impose over the output tokens: we don't want tokens to mix characters from different categories (as in text, digits, whitespace of others).

The reason why is that if we have some very common token like "for" and we tokenize it early on, we'll find many occurrences of that same token followed by different stuff, such as "for.", "for?", or "for!". This will create many tokens that encode the same stuff, which is not very useful.

We can use the `regex` library, which is a more powerful version of the builtin `re` library that defines groups for unicode subsets. While the `\w` in regex matches any letter or digit, the `\p{L}` and `\p{N}` match only letters and digits respectively. Plus, the `\p{Z}` matches any kind of whitespace.

In [5]:
split = regex_compile(r'\p{Z}?(?:\p{L}+|\p{N}+)|\p{Z}+|.').findall
split('Let\'s see how this w0rks!')

['Let', "'", 's', ' see', ' how', ' this', ' w', '0', 'rks', '!']

Now we have all the required ingredients to implement our BPE tokenization method. Any tokenizer would have the following methods:
* `train`: which uses some given dataset to learn the tokenization rules.
* `encode`: which converts a given text into a list of tokens.
* `decode`: which converts a list of tokens into a text.

Let's start by implementing the `train` method of our BPE tokenizer.

In [6]:
def train_tokenizer(text, max_merges=2000):
	pieces = split(text)
	elems = [list(text.encode('utf-8')) for text in pieces]
	start = time()
	next_replacement = 256
	vocabulary = [[i] for i in range(next_replacement)]
	merges = {}
	for m in range(max_merges):
		# find the most frequent pair of elements across all pieces
		count = {}
		for piece in elems:
			for e1, e2 in zip(piece[:-1], piece[1:]):
				count[(e1, e2)] = count.get((e1, e2), 0) + 1
		pair, freq = max(count.items(), key=lambda x: x[1])
		# if the pair is not frequent enough, stop
		if freq == 1:
			print('Stopping early, no pair found more than once.')
			break
		# merge the pair
		vocabulary.append(vocabulary[pair[0]] + vocabulary[pair[1]])
		merges[pair] = next_replacement
		for i, piece in enumerate(elems):
			elems[i] = merge(piece, pair, next_replacement)
		next_replacement += 1
		# print progress
		if m % (max_merges // 10) == 0:
			remaining = (time() - start) * (max_merges - m) / (m + 1)
			remaining = f'{remaining//3600:02.0f}:{remaining%3600//60:02.0f}:{remaining%60:02.0f}'
			print(f'{m:4} merges done, {remaining} remaining, compression ratio 1:{len(text) / sum(map(len, elems)):.2f}')
	return elems, vocabulary, merges

compressed_text, vocabulary, merges = train_tokenizer(text)
print(f'Final compression ratio 1:{len(text) / sum(map(len, compressed_text)):.2f}')

   0 merges done, 03:54:30 remaining, compression ratio 1:1.01
 200 merges done, 02:30:31 remaining, compression ratio 1:1.98
 400 merges done, 02:05:13 remaining, compression ratio 1:2.26
 600 merges done, 01:45:46 remaining, compression ratio 1:2.43
 800 merges done, 01:28:34 remaining, compression ratio 1:2.56
1000 merges done, 01:12:31 remaining, compression ratio 1:2.66
1200 merges done, 00:57:16 remaining, compression ratio 1:2.75
1400 merges done, 00:42:28 remaining, compression ratio 1:2.82
1600 merges done, 00:28:03 remaining, compression ratio 1:2.88
1800 merges done, 00:13:56 remaining, compression ratio 1:2.94
Final compression ratio 1:2.99


The `train` method has gone over the dataset looking for common pair of elements, and outputs the following structures:
* `compressed_text`: The text encoded using our tokenization rules, which can be used to compute the compression ratio or to find further merges.
* `vocabulary`: A dictionary that maps each token to the set of bytes it represents.
* `merges`: A dictinary that maps each pair of tokens to the new token that replaces them.

Both the `vocabulary` and `merges` dictionaries required to implement the `encode` and `decode` methods. Let's implement them and see if they work.

In [7]:
def encode(text, merges):
	res = list(text.encode('utf-8'))
	for pair, replacement in merges.items():
		res = merge(res, pair, replacement)
	return res

def decode(elems, vocabulary):
	res = b''.join(bytes(vocabulary[e]) for e in elems)
	return res.decode('utf-8')

text = 'this is a test to see if it works °å^○ⁿ·'
encoded = encode(text, merges)
decoded = decode(encoded, vocabulary)
print(f'Original text: {text}')
print(f'Encoded text: {encoded}')
print(f'Decoded text: {decoded}')

Original text: this is a test to see if it works °å^○ⁿ·
Encoded text: [116, 104, 499, 385, 115, 265, 285, 271, 116, 380, 304, 101, 385, 102, 385, 116, 32, 119, 277, 107, 115, 32, 194, 176, 195, 165, 94, 226, 151, 139, 226, 129, 191, 194, 183]
Decoded text: this is a test to see if it works °å^○ⁿ·


And that's it! We have implemented a simple BPE tokenizer and trained it to create 2000 new tokens from the dataset of Spanish novels.

Note that generating a tokenizer is a slow process (it took us 4 hours in a M2 Macbook Pro) even on a tiny dataset like ours. Doing this at large scale requires figuring out more complex processes that parallelize the search for the most common pair of elements and way more computational resources. This is why most people use pre-trained tokenizers such as [OpenAI's tiktoken](https://github.com/openai/tiktoken) or Google's [SentencePiece](https://github.com/google/sentencepiece).

The biggest problem with these tokenizers is that they are mostly trained over English text, which means that they are not very good at tokenizing text in other languages (including programming languages). Plus, tokenization is at the root of many AI problems such as:
* Not being able to perform simple arithmetic like 123 + 456, because maybe the tokenizer splits 123 as "12" and "3".
* Not being able to perform simple string operation like counting, spelling or reversing sentences.

It is one of the most hated pieces of the current status of AI, and ideally having alternatives to attention mechanisms that can handle arbitrarily longer sequences and being able to use character-level tokenization would be a huge step forward. However, this is the best we got so far.